In [30]:
import numpy as np

def log_cosh_loss(y_true, y_pred):
    y_true, y_pred = np.array(y_true, dtype=np.float64), np.array(y_pred, dtype=np.float64)
    
    # 動態縮放因子，僅根據 y_true 計算，防止溢位
    data_range = np.max(np.abs(y_true))
    if data_range > 1000000:
        scaling_factor = 1e-7  # 調整縮放因子
    elif data_range > 10000:
        scaling_factor = 1e-5  # 調整縮放因子
    elif data_range > 100:
        scaling_factor = 1e-3  # 調整縮放因子
    else:
        scaling_factor = 1.0
    
    def _log_cosh(x):
        # 使用數值穩定的公式來避免溢位
        return np.where(np.abs(x) > 20, np.abs(x) - np.log(2), np.log(np.cosh(x)))
    
    # 使用縮放後的差異值來計算
    loss = _log_cosh(scaling_factor * (y_pred - y_true))
    
    # 將損失除以縮放因子以保持量級
    return np.mean(loss) / scaling_factor

A = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7]
B = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 700]
print(log_cosh_loss(A, B))

A = [1, 2, 3, 4, 5, 6, 7]
B = [1, 2, 3, 4, 5, 6, 70]
print(log_cosh_loss(A, B))


16.100159367893955
8.900978974205723


In [31]:
def success_rate(A, B):
    # 檢查兩個列表的長度是否相等，若不相等則返回 0
    if len(A) != len(B):
        return 0

    # 計算相同元素的個數
    correct_count = sum(1 for a, b in zip(A, B) if a == b)
    # 計算完全正確的比例
    rate = correct_count / len(A)
    
    return rate

# 測試範例
A = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7]
B = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 700]
print(success_rate(A, B))  # 輸出：0.9761904761904762

A = [1, 2, 3, 4, 5, 6, 7]
B = [1, 2, 3, 4, 5, 6, 70]
print(success_rate(A, B))  # 輸出：0.8571428571428571


0.9767441860465116
0.8571428571428571


In [32]:
# Manually extract and process the nested 'output' dictionary from each record
import json
import pandas as pd

# Load the JSONL file again
file_path = './data/finetuning_training_data.jsonl'

# Read the JSONL file into a list of dictionaries
data = [json.loads(line) for line in open(file_path, 'r', encoding='utf-8')]

# Extract the 'output' part of each record and create a new DataFrame
output_data = [record['output'] for record in data]
df = pd.DataFrame(output_data)

# Define the string and integer fields
string_fields = ["事故日期", "事發經過", "事故車出廠日期", "傷勢", "職業", "折舊方法", "被告肇責"]
int_fields = ["塗裝", "工資", "烤漆", "鈑金", "耐用年數", "修車費用", "醫療費用", "賠償金額總額", "保險給付金額", "居家看護天數", "居家看護費用", "每日居家看護金額"]

# Calculate average string length for string fields
string_averages = df[string_fields].applymap(lambda x: len(str(x)) if pd.notnull(x) else 0).mean()

# Calculate distribution (min, max, mean) for int fields
int_distributions = df[int_fields].apply(pd.to_numeric, errors='coerce').describe().loc[['min', 'max', 'mean']]

string_averages, int_distributions


/var/folders/kc/jbnf20ls6hl2qsfkbcc8pb600000gn/T/ipykernel_37381/2634073042.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_averages = df[string_fields].applymap(lambda x: len(str(x)) if pd.notnull(x) else 0).mean()


(事故日期        0.986672
 事發經過       17.992332
 事故車出廠日期     0.452437
 傷勢          1.014972
 職業          0.063173
 折舊方法        0.284097
 被告肇責        0.057331
 dtype: float64,
             塗裝            工資           烤漆       鈑金      耐用年數          修車費用  \
 min    4860.00    122.000000   100.000000    800.0  3.000000    233.000000   
 max   11000.00  59262.000000  8682.000000  26000.0  5.000000  76532.000000   
 mean   8333.75   5320.107143  4320.666667  10665.5  4.623306  12110.166667   
 
              醫療費用         賠償金額總額  保險給付金額      居家看護天數    居家看護費用     每日居家看護金額  
 min    100.000000     683.000000  2515.0    6.000000   26000.0   714.000000  
 max   5063.000000  106931.000000  2515.0  150.000000  120000.0  2200.000000  
 mean  1107.611111   26176.857143  2515.0   47.964286   71000.0  1316.333333  )

In [4]:
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def chinese_tokenizer(text):
    return jieba.lcut(text)

def calculate_average_cosine_similarity(text_list_1, text_list_2):
    vectorizer = CountVectorizer(stop_words='english')
    total_similarity = 0
    valid_pairs = 0

    for text1, text2 in zip(text_list_1, text_list_2):
        text1 = str(text1)
        text2 = str(text2)
        
        if text1.strip() == '' and  text2.strip() == '':
            total_similarity += 1
            valid_pairs += 1
            
        elif text1.strip() == '' and  text2.strip() != '':
            total_similarity += 0
            valid_pairs += 1
        
        elif text1.strip() != '' and  text2.strip() == '':
            total_similarity += 0
            valid_pairs += 1
    
        else:  # 檢查非空字串
            corpus = [text1, text2]
            vectors = vectorizer.fit_transform(corpus)
            if vectors.shape[1] > 0:  # 檢查詞彙表是否為空
                similarity = cosine_similarity(vectors)
                total_similarity += similarity[0][1]
                valid_pairs += 1
                
    if valid_pairs == 0:
        return 0  # 如果沒有有效的字串對，返回0或其他預設值

    average_similarity = total_similarity / valid_pairs
    return average_similarity

A=['', '', '', '', '', '便利商店員工', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '駕駛', '', '', '', '', '', '', '', '', '', '', '', '', '郵局投遞士', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '工程師', '', '服務業', '', '', '', '', '', '', '家翊公司', '', '', '', '', '', '', '汽車維修', '', '', '', '', '', '理事長秘書', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '服務業', '', '', '', '', '', '', '家管', '', '', '機械設備進口貿易', '', '英文家教', '', '', '', '', '專櫃銷售員', '家庭主婦', '電器行', '', '', '', '家庭主婦', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '販賣水果', '', '', '', '', '', '', '', '中鋼員工', '', '', '', '', '', '', '', '', '計程車', '', '', '', '', '', '', '', '', '', '', '南臺科技大學擔任辦事員', '', '公司負責人', '', '', '打0工', '', '', '', '', '', '', '', '', '', '', '', '']
B=['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '損失', '', '', '', '', '', '', '', '', '天，且原告確已因本件侵權而實際支出，尚有誤會，不應准許。(2)精神慰撫金：經審酌「放屁」、「怪不得你這種人只能做司機」、「他媽的」、「幹你老師咧」等語，均屬負面用語，足以減低他人之社會評價，並對司機這種對社會有貢獻之職業，以自身之優越社經地位主觀加以貶抑，並加諸於原告，情節顯屬重大，經依最高法院', '', '', '', '', '', '', '', '', '能力，而可扶養原告蘇連得。而原告蘇連得名下雖有坐落屏東縣○○市○○段', '', '', '', '', '', '', '', '', '', '，開始享受與原告洪明通及洪仁吉、洪仁風、洪諺邦全家天倫之樂時，因本件車禍竟造成洪李美當場死亡結果，造成原告洪明通、洪仁吉、洪仁風及洪諺邦精神極大痛苦，身心遭受極大之創傷及痛苦已超乎1般常人可忍受之程度。被告於肇事後，毫無悔意，且事發至今，被告亦未曾表示誠意，給予原告等人任何喪葬費或任何表示歉意之意思，並於治喪期間糾眾至喪家，犯後態度極度惡劣已造成原告家屬2次傷害。爰依民法第', '', '', '', '損失', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '、危害之嚴重性、被害法益之輕重、防範避免危害之代價、與公共利益之關係、資料來源之可信度、查證之難易等，而有所不同；後者乃行為人表示自己之見解或立場，屬主觀價值判斷之範疇，無真實與否可言，行為人對於可受公評之事，如係善意發表適當評論，固不具違法性，然行為人倘對於未能確定之事實，使用偏激不堪之言詞而為意見表達，足以貶損他人在社會上之評價，仍屬侵害他人之名譽權，應負侵權行為之損害賠償責任，又依社會通念為客觀之綜合評價，倘該行為傷及被害人主觀之情感並對被害人社會之客觀評價產生不良影響，即屬名譽之侵害又民法上名譽權之侵害非即與刑法之誹謗罪相同，名譽有無受損害，應以社會上對個人評價是否貶損作為判斷之依據，苟其行為足以使他人在社會上之評價受到貶損，不論其為故意或過失，均可構成侵權行為，其行為不以廣佈於社會為必要，僅使第3人知悉其事，亦足當之民法上，若已將足以毀損他人名譽之事表白於特定第3人，縱未至公然侮辱之程度，且無散布於眾之意圖，亦應認係名譽之侵害，蓋既對於第3人表白足以毀損他人名譽之事，則其人之社會評價，不免因而受有貶損。(最高法院', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '，名下財產有數筆不動產等語，而依卷附本院依職權查詢之原告之稅務電子閘門財產所得調件明細表', '', '', '', '', '', '', '', '', '', '', '', '之損失', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '之被告乙○○發展超越社交禮儀、逾越分際之外遇行為。被告乙○○更時常手寫內容曖昧之卡片、用紙摺成愛心圖案送給被告甲○○。原告於', '', '', '', '', '', '', '', '', '', '', '', '損失', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
print("ANS=", calculate_average_cosine_similarity(A, B))


ANS= 0.0


In [46]:
from processed_to_format import basicPrompt, advancedPrompt, oneShotPrompt
print("token of basicPrompt:", len(basicPrompt()))
print("token of advancedPrompt:", len(advancedPrompt()))
print("token of oneShotPrompt:", len(oneShotPrompt()))

token of basicPrompt: 200
token of advancedPrompt: 584
token of oneShotPrompt: 2772
